<a href="https://colab.research.google.com/github/preethishp/ExerciseClassificationAndFormCorrection/blob/main/AIBasedExerciseRecognitionAndFormCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"preethishp","key":"40ff96159a26942305cddf86dca2d421"}'}

In [ ]:
!mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Download & unzip the dataset (hasyimabdillah/workoutfitness-video)
!kaggle datasets download -d hasyimabdillah/workoutfitness-video -p /content/workout --unzip

Dataset URL: https://www.kaggle.com/datasets/hasyimabdillah/workoutfitness-video
License(s): CC-BY-NC-SA-4.0
100% 4.31G/4.32G [00:47<00:00, 149MB/s]
100% 4.32G/4.32G [00:47<00:00, 98.4MB/s]


In [1]:
!ls -altr

total 16
drwxr-xr-x 4 root root 4096 Aug 28 13:42 .config
drwxr-xr-x 1 root root 4096 Aug 28 13:42 .
drwxr-xr-x 1 root root 4096 Aug 28 13:43 sample_data
drwxr-xr-x 1 root root 4096 Aug 31 20:29 ..


In [ ]:
!pip uninstall -y mediapipe protobuf || true
!pip install -q ultralytics opencv-python tqdm
import torch, cv2, numpy as np
from ultralytics import YOLO
print("Ultralytics ready. Torch:", torch.__version__)

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Ultralytics ready. Torch: 2.8.0+cu126


In [ ]:
from pathlib import Path
from tqdm import tqdm

# Load pose model
pose_model = YOLO('yolov8n-pose.pt')

VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v", ".webm"}
EVERY_N_FRAMES = 2
SEQ_MIN_FRAMES = 24

LEFT_HIP, RIGHT_HIP = 11, 12
FEATS = 17 * 2  # (x,y) for 17 kpts => 34

def extract_pose_sequence_yolo(video_path, every_n=EVERY_N_FRAMES):
    cap = cv2.VideoCapture(str(video_path))
    seq = []
    fidx = 0
    while cap.isOpened():
        ok, frame = cap.read()
        if not ok: break
        if fidx % every_n == 0:
            # Inference (return keypoints)
            results = pose_model.predict(frame, verbose=False)
            if not results or len(results[0].keypoints) == 0:
                fidx += 1; continue

            kps = results[0].keypoints  # N persons × 17 × 3 (x,y,conf)
            # pick the person with the highest mean keypoint confidence
            confs = kps.conf.squeeze(-1) if kps.conf.ndim == 3 else kps.conf  # (N,17)
            person_idx = torch.argmax(confs.mean(dim=1)).item()
            xy = kps.xy[person_idx].cpu().numpy()  # (17,2) absolute pixels

            # normalize by mid-hip to be camera-position invariant
            lh, rh = xy[LEFT_HIP], xy[RIGHT_HIP]
            origin = (lh + rh) / 2.0
            xy_norm = (xy - origin)

            seq.append(xy_norm.reshape(-1))  # (34,)
        fidx += 1
    cap.release()
    if len(seq) == 0:
        return np.zeros((0, FEATS), dtype=np.float32)
    return np.asarray(seq, dtype=np.float32)  # (T,34)

In [ ]:
ROOT = Path("/content/workout")
OUT  = Path("/content/pose_seq_yolo")
OUT.mkdir(parents=True, exist_ok=True)

items = []
for p in ROOT.rglob("*"):
    if p.is_file() and p.suffix.lower() in VIDEO_EXTS:
        label = p.parent.name
        items.append((p, label))

labels = sorted({lbl for _, lbl in items})
label2id = {lbl:i for i,lbl in enumerate(labels)}
print("Classes:", labels)

index = []
for path, label in tqdm(items, desc="Pose->seq (YOLO)"):
    try:
        arr = extract_pose_sequence_yolo(path)
        if len(arr) < SEQ_MIN_FRAMES:
            continue
        out = OUT / f"{label2id[label]:02d}__{path.stem}.npy"
        np.save(out, arr)
        index.append((out, label2id[label]))
    except Exception as e:
        print("Skipping:", path, e)

len(index)

Classes: []


Pose->seq (YOLO): 0it [00:00, ?it/s]


0

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"preethishp","key":"40ff96159a26942305cddf86dca2d421"}'}

In [ ]:
# Reinstall protobuf (compatible version for Kaggle CLI)
!pip install -q "protobuf<4.0.0"

# Reinstall kaggle to make sure dependencies are correct
!pip install -q --force-reinstall kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list -s workout | head -n 20

ref                                                              title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  ------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
hasyimabdillah/workoutfitness-video                              Workout/Exercises Video                            4641127680  2023-03-10 15:52:07.917000           6530         59  0.875            
hasyimabdillah/workoutexercises-images                           Workout/Exercise Images                             857735926  2023-03-06 15:52:45.550000           2378         48  0.875            
valakhorasani/gym-members-exercise-dataset                       Gym Members Exercise Dataset                            22093  2024-10-06 11:27:38.750000          40822        461  1.0              


In [ ]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video -p /content/data --unzip

Dataset URL: https://www.kaggle.com/datasets/hasyimabdillah/workoutfitness-video
License(s): CC-BY-NC-SA-4.0
100% 4.31G/4.32G [01:19<00:00, 43.1MB/s]
100% 4.32G/4.32G [01:19<00:00, 58.1MB/s]


In [ ]:
from pathlib import Path
from collections import Counter

def scan(root=Path("/content/data")):
    exts = Counter(p.suffix.lower() for p in root.rglob("*") if p.is_file())
    vids = [p for p in root.rglob("*") if p.is_file() and p.suffix.lower() in {".mp4",".mov",".avi",".mkv",".m4v",".webm"}]
    print("Root:", root)
    print("File extensions (top):", exts.most_common(10))
    print("Video count:", len(vids))
    for p in vids[:10]:
        print(" •", p)
    return root, vids

ROOT, vids = scan(Path("/content/data"))

Root: /content/data
File extensions (top): [('.mp4', 590), ('.mov', 62)]
Video count: 652
 • /content/data/hammer curl/hammer curl_15.mp4
 • /content/data/hammer curl/hammer curl_6.MOV
 • /content/data/hammer curl/hammer curl_11.mp4
 • /content/data/hammer curl/hammer curl_18.mp4
 • /content/data/hammer curl/hammer curl_4.MOV
 • /content/data/hammer curl/hammer curl_2.MOV
 • /content/data/hammer curl/hammer curl_14.mp4
 • /content/data/hammer curl/hammer curl_5.MOV
 • /content/data/hammer curl/hammer curl_10.mp4
 • /content/data/hammer curl/hammer curl_17.mp4


In [ ]:

from pathlib import Path
ROOT = Path("/content/data")
OUT  = Path("/content/pose_seq_yolo"); OUT.mkdir(parents=True, exist_ok=True)
VIDEO_EXTS       = {".mp4",".mov",".avi",".mkv",".m4v",".webm"}
EVERY_N_FRAMES   = 2
SEQ_MIN_FRAMES   = 24
SEQ_LEN          = 60
FEATS            = 34
BATCH_SIZE       = 8

In [ ]:

import re
items = []
for p in ROOT.rglob("*"):
    if p.is_file() and p.suffix.lower() in VIDEO_EXTS:
        label = p.parent.name
        items.append((p, label))

labels = sorted({lbl for _, lbl in items})
label2id = {lbl:i for i,lbl in enumerate(labels)}
print(f"Discovered {len(items)} videos across {len(labels)} classes:")
print(labels[:50])
assert len(items) > 0, "No videos found – double-check ROOT path or unzip."

Discovered 652 videos across 22 classes:
['barbell biceps curl', 'bench press', 'chest fly machine', 'deadlift', 'decline bench press', 'hammer curl', 'hip thrust', 'incline bench press', 'lat pulldown', 'lateral raise', 'leg extension', 'leg raises', 'plank', 'pull Up', 'push-up', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat', 't bar row', 'tricep Pushdown', 'tricep dips']


In [ ]:
import cv2, numpy as np, torch
from ultralytics import YOLO
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

pose_model = YOLO("yolov8n-pose.pt")
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
pose_model.to(DEVICE)

if torch.cuda.is_available():
    pose_model.model.half()

LEFT_HIP, RIGHT_HIP = 11, 12
LEFT_SHO, RIGHT_SHO = 5, 6

def _preprocess_frame(frame, target=448):
    h, w = frame.shape[:2]
    if max(h, w) > target:
        scale = target / max(h, w)
        frame = cv2.resize(frame, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)
    return frame

def extract_pose_sequence_yolo_batched(video_path, every_n=3, batch_size=32, imgsz=448, conf=0.25):
    cap = cv2.VideoCapture(str(video_path))
    frames = []
    fidx = 0
    while cap.isOpened():
        ok, frame = cap.read()
        if not ok:
            break
        if fidx % every_n == 0:
            frames.append(_preprocess_frame(frame, imgsz))
        fidx += 1
    cap.release()

    if not frames:
        return np.zeros((0, 34), dtype=np.float32)

    seq = []
    # Run in batches on GPU
    for i in range(0, len(frames), batch_size):
        batch = frames[i:i+batch_size]
        results = pose_model.predict(
            batch,
            imgsz=imgsz,
            device=DEVICE,
            conf=conf,
            max_det=1,
            verbose=False
        )
        for res in results:
            # If nothing detected, skip
            if not hasattr(res, "keypoints") or res.keypoints is None or len(res.keypoints) == 0:
                continue
            # keypoints: (N, 17, 3) but N=1 with max_det=1
            xy = res.keypoints.xy[0].float().cpu().numpy()  # (17,2)

            # origin = mid-hip; scale = distance to mid-shoulder
            origin = (xy[LEFT_HIP] + xy[RIGHT_HIP]) / 2.0
            shoulder_mid = (xy[LEFT_SHO] + xy[RIGHT_SHO]) / 2.0
            scale = np.linalg.norm(shoulder_mid - origin)
            if scale < 1e-6:
                scale = 1.0

            xy_norm = (xy - origin) / scale
            seq.append(xy_norm.reshape(-1))  # (34,)
    if not seq:
        return np.zeros((0, 34), dtype=np.float32)
    return np.asarray(seq, dtype=np.float32)
# Build items as (path, y_id) instead of (path, label_str)
label2id = {lbl:i for i, lbl in enumerate(labels)}
items_id = [(p, label2id[lbl]) for (p, lbl) in items]


import numpy as np
from pathlib import Path

def process_one(args):
    path, y_id = args  # y_id is int now
    arr = extract_pose_sequence_yolo_batched(
        path, every_n=EVERY_N_FRAMES, batch_size=32, imgsz=448, conf=0.25
    )
    if len(arr) >= SEQ_MIN_FRAMES:
        out = OUT / f"{y_id:02d}__{path.stem}.npy"
        np.save(out, arr)
        return (out, y_id)
    return None

with Pool(min(4, cpu_count())) as pool:
    results = list(tqdm(pool.imap(process_one, items_id, chunksize=2), total=len(items_id)))
index = [r for r in results if r is not None]
print("Cached sequences:", len(index))

100%|██████████| 652/652 [1:59:58<00:00, 11.04s/it]


Cached sequences: 630


In [ ]:
from pathlib import Path
import pandas as pd
import json
import re


SNAP_DIR = OUT
META_DIR = SNAP_DIR
INDEX_CSV = META_DIR / "index.csv"
LABELS_JSON = META_DIR / "labels.json"

def snapshot_cache(out_dir: Path, labels=None, label2id=None):
    out_dir = Path(out_dir)
    npys = sorted(out_dir.glob("*.npy"))
    entries = []
    for f in npys:
        m = re.match(r"^(\d+?)__.+?\.npy$", f.name)
        if not m:
            continue
        y_id = int(m.group(1))
        entries.append({"seq_path": str(f), "label_id": y_id})

    df = pd.DataFrame(entries)
    df.to_csv(out_dir / "index.csv", index=False)


    meta = {}
    if labels is not None:
        meta["labels"] = list(labels)
    if label2id is not None:

        meta["label2id"] = {str(k): int(v) for k, v in label2id.items()}
    if meta:
        with open(out_dir / "labels.json", "w") as f:
            json.dump(meta, f, indent=2)

    print(f"Snapshot saved: {out_dir/'index.csv'}", end="")
    if meta:
        print(f" and {out_dir/'labels.json'}")
    else:
        print(" (no labels.json provided)")

try:
    snapshot_cache(SNAP_DIR, labels=labels, label2id=label2id)
except NameError:
    snapshot_cache(SNAP_DIR, labels=None, label2id=None)

✅ Snapshot saved: /content/pose_seq_yolo/index.csv and /content/pose_seq_yolo/labels.json


In [ ]:

import numpy as np, torch, random
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = len(labels)

def pad_or_center_trim(x, T=SEQ_LEN):
    t = len(x)
    if t == T: return x
    if t > T:
        s = (t - T)//2
        return x[s:s+T]
    pad = np.zeros((T - t, x.shape[1]), dtype=x.dtype)
    return np.concatenate([x, pad], axis=0)

class PoseSeqDS(Dataset):
    def __init__(self, entries): self.entries = entries
    def __len__(self): return len(self.entries)
    def __getitem__(self, i):
        p, y = self.entries[i]
        x = np.load(p)
        x = pad_or_center_trim(x, SEQ_LEN)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

random.seed(42)
bylab = defaultdict(list)
for p,y in index: bylab[y].append((p,y))
train = []; val = []; test = []
for y, L in bylab.items():
    random.shuffle(L)
    n=len(L); ntest=max(1,int(0.15*n)); nval=max(1,int(0.15*n))
    test += L[:ntest]; val += L[ntest:ntest+nval]; train += L[ntest+nval:]

train_dl = DataLoader(PoseSeqDS(train), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_dl   = DataLoader(PoseSeqDS(val),   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_dl  = DataLoader(PoseSeqDS(test),  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

len(train), len(val), len(test)

(460, 85, 85)

In [ ]:
# Initial LSTM model + training
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, feats=FEATS, hidden=128, layers=1, num_classes=NUM_CLASSES, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(feats, hidden, num_layers=layers, batch_first=True, dropout=0.0 if layers==1 else dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden, num_classes)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = self.dropout(h[-1])
        return self.fc(h)

model = LSTMClassifier().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def eval_loader(net, loader):
    net.eval()
    total, correct, loss_sum = 0, 0, 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = net(xb)
            loss_sum += criterion(logits, yb).item() * yb.size(0)
            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
    return loss_sum/max(total,1), correct/max(total,1)

EPOCHS = 25
for ep in range(1, EPOCHS+1):
    model.train()
    running, batches = 0.0, 0
    for xb, yb in train_dl:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        loss = criterion(model(xb), yb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        running += loss.item(); batches += 1
    tr = running/max(batches,1)
    vl, va = eval_loader(model, val_dl)
    print(f"Epoch {ep:02d} | train={tr:.4f} | val_loss={vl:.4f} | val_acc={va:.3f}")

tl, ta = eval_loader(model, test_dl)
print(f"🏁 Test: loss={tl:.4f} | acc={ta:.3f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 01 | train=2.9255 | val_loss=2.5670 | val_acc=0.212
Epoch 02 | train=2.4445 | val_loss=2.2968 | val_acc=0.400
Epoch 03 | train=2.3093 | val_loss=2.2186 | val_acc=0.353
Epoch 04 | train=2.1133 | val_loss=1.9115 | val_acc=0.400
Epoch 05 | train=1.8887 | val_loss=2.1875 | val_acc=0.282
Epoch 06 | train=2.0765 | val_loss=1.8899 | val_acc=0.376
Epoch 07 | train=1.9775 | val_loss=1.9106 | val_acc=0.365
Epoch 08 | train=1.7340 | val_loss=1.6615 | val_acc=0.482
Epoch 09 | train=1.9142 | val_loss=1.8131 | val_acc=0.424
Epoch 10 | train=1.7760 | val_loss=1.8180 | val_acc=0.459
Epoch 11 | train=1.8588 | val_loss=2.4749 | val_acc=0.259
Epoch 12 | train=1.9032 | val_loss=1.7026 | val_acc=0.424
Epoch 13 | train=1.6277 | val_loss=1.6885 | val_acc=0.435
Epoch 14 | train=1.5610 | val_loss=1.5102 | val_acc=0.506
Epoch 15 | train=1.7898 | val_loss=1.7554 | val_acc=0.447
Epoch 16 | train=1.5107 | val_loss=1.4701 | val_acc=0.541
Epoch 17 | train=1.5093 | val_loss=1.6090 | val_acc=0.576
Epoch 18 | tra

In [ ]:
import numpy as np

# COCO indices
LHIP, RHIP, LKNEE, RKNEE, LANK, RANK = 11,12,13,14,15,16
LSHO, RSHO, LELB, RELB, LWR, RWR     = 5,6,7,8,9,10

def angle_deg(a, b, c):
    ba = a - b; bc = c - b
    den = (np.linalg.norm(ba)*np.linalg.norm(bc))
    if den == 0: return 180.0
    cosv = np.clip(np.dot(ba,bc)/den, -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))

def clip_stats(seq_xy):  # (T,34)
    stats = {"knee_r":[], "knee_l":[], "elbow_r":[], "elbow_l":[]}
    for t in range(len(seq_xy)):
        xy = seq_xy[t].reshape(17,2)
        stats["knee_r"].append(angle_deg(xy[RHIP], xy[RKNEE], xy[RANK]))
        stats["knee_l"].append(angle_deg(xy[LHIP], xy[LKNEE], xy[LANK]))
        stats["elbow_r"].append(angle_deg(xy[RSHO], xy[RELB], xy[RWR]))
        stats["elbow_l"].append(angle_deg(xy[LSHO], xy[LELB], xy[LWR]))
    return {k: {"min":float(np.min(v)), "p90":float(np.percentile(v,90))} for k,v in stats.items()}

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

def collect_preds(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in loader:
            p = model(xb.to(DEVICE)).argmax(1).cpu().numpy()
            preds.append(p); trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

preds_val, y_val_np = collect_preds(model, val_dl)
print(classification_report(y_val_np, preds_val, target_names=labels))
print(confusion_matrix(y_val_np, preds_val))

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


                     precision    recall  f1-score   support

barbell biceps curl       0.55      0.67      0.60         9
        bench press       0.75      0.43      0.55         7
  chest fly machine       0.67      0.50      0.57         4
           deadlift       0.50      0.75      0.60         4
decline bench press       0.00      0.00      0.00         1
        hammer curl       0.00      0.00      0.00         2
         hip thrust       1.00      1.00      1.00         2
incline bench press       0.60      0.75      0.67         4
       lat pulldown       0.50      0.71      0.59         7
      lateral raise       0.29      0.40      0.33         5
      leg extension       0.50      0.67      0.57         3
         leg raises       0.00      0.00      0.00         3
              plank       0.00      0.00      0.00         1
            pull Up       0.60      1.00      0.75         3
            push-up       0.78      0.88      0.82         8
  romanian deadlift    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch.nn as nn

class Chomp1d(nn.Module):
    def __init__(self, c): super().__init__(); self.c=c
    def forward(self, x): return x[:, :, :-self.c].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_in, n_out, k, dilation, padding, dropout=0.2):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv1d(n_in, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
            nn.Conv1d(n_out, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
        )
        self.down = nn.Conv1d(n_in, n_out, 1) if n_in!=n_out else nn.Identity()
    def forward(self, x):
        out = self.seq(x); return out + self.down(x)

class TCNClassifier(nn.Module):
    def __init__(self, feats=FEATS, channels=[64,64,128], k=5, dropout=0.2, num_classes=NUM_CLASSES):
        super().__init__()
        layers=[]; n_in=feats
        for i, ch in enumerate(channels):
            dil=2**i; pad=(k-1)*dil
            layers.append(TemporalBlock(n_in, ch, k, dil, pad, dropout))
            n_in=ch
        self.tcn = nn.Sequential(*layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc   = nn.Linear(n_in, num_classes)
    def forward(self, x):               # x: (B,T,F)
        x = x.transpose(1,2)            # (B,F,T)
        y = self.tcn(x)                 # (B,C,T)
        y = self.pool(y).squeeze(-1)    # (B,C)
        return self.fc(y)

model = TCNClassifier().to(DEVICE)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def eval_loader(net, loader):
    net.eval()
    total, correct, loss_sum = 0, 0, 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = net(xb)
            loss_sum += criterion(logits, yb).item() * yb.size(0)
            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
    return loss_sum/max(total,1), correct/max(total,1)

EPOCHS = 25
for ep in range(1, EPOCHS+1):
    model.train()
    running, batches = 0.0, 0
    for xb, yb in train_dl:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        loss = criterion(model(xb), yb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        running += loss.item(); batches += 1
    tr = running/max(batches,1)
    vl, va = eval_loader(model, val_dl)
    print(f"Epoch {ep:02d} | train={tr:.4f} | val_loss={vl:.4f} | val_acc={va:.3f}")

tl, ta = eval_loader(model, test_dl)
print(f"🏁 Test: loss={tl:.4f} | acc={ta:.3f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 01 | train=0.2499 | val_loss=1.2686 | val_acc=0.765
Epoch 02 | train=0.3573 | val_loss=1.3417 | val_acc=0.765
Epoch 03 | train=0.3367 | val_loss=1.4739 | val_acc=0.765
Epoch 04 | train=0.2957 | val_loss=1.3478 | val_acc=0.753
Epoch 05 | train=0.1396 | val_loss=1.2850 | val_acc=0.788
Epoch 06 | train=0.1262 | val_loss=1.3458 | val_acc=0.776
Epoch 07 | train=0.1103 | val_loss=1.2177 | val_acc=0.765
Epoch 08 | train=0.1434 | val_loss=1.3165 | val_acc=0.800
Epoch 09 | train=0.1488 | val_loss=1.7329 | val_acc=0.765
Epoch 10 | train=0.3369 | val_loss=1.4453 | val_acc=0.753
Epoch 11 | train=0.3831 | val_loss=1.4063 | val_acc=0.776
Epoch 12 | train=0.3813 | val_loss=1.4722 | val_acc=0.706
Epoch 13 | train=0.3901 | val_loss=1.4782 | val_acc=0.706
Epoch 14 | train=0.2430 | val_loss=1.2312 | val_acc=0.753
Epoch 15 | train=0.0943 | val_loss=1.4995 | val_acc=0.812
Epoch 16 | train=0.0489 | val_loss=1.5025 | val_acc=0.776
Epoch 17 | train=0.0386 | val_loss=1.4289 | val_acc=0.800
Epoch 18 | tra

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

def collect_preds(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in loader:
            p = model(xb.to(DEVICE)).argmax(1).cpu().numpy()
            preds.append(p); trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

preds_val, y_val_np = collect_preds(model, val_dl)
print(classification_report(y_val_np, preds_val, target_names=labels))
print(confusion_matrix(y_val_np, preds_val))

                     precision    recall  f1-score   support

barbell biceps curl       0.82      1.00      0.90         9
        bench press       0.62      0.71      0.67         7
  chest fly machine       0.75      0.75      0.75         4
           deadlift       1.00      0.75      0.86         4
decline bench press       0.00      0.00      0.00         1
        hammer curl       1.00      1.00      1.00         2
         hip thrust       1.00      0.50      0.67         2
incline bench press       1.00      0.50      0.67         4
       lat pulldown       0.71      0.71      0.71         7
      lateral raise       0.71      1.00      0.83         5
      leg extension       0.75      1.00      0.86         3
         leg raises       1.00      0.33      0.50         3
              plank       1.00      1.00      1.00         1
            pull Up       1.00      0.67      0.80         3
            push-up       1.00      1.00      1.00         8
  romanian deadlift    

In [ ]:
%%writefile train_tcn.py
#!/usr/bin/env python3
import argparse, os, json, random, re
from pathlib import Path
from collections import defaultdict, Counter

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# =========================
# Repro
# =========================
def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# =========================
# Sequence utils
# =========================
def pad_or_center_trim(x: np.ndarray, T: int) -> np.ndarray:
    t = len(x)
    if t == T: return x
    if t > T:
        s = (t - T)//2
        return x[s:s+T]
    pad = np.zeros((T - t, x.shape[1]), dtype=x.dtype)
    return np.concatenate([x, pad], axis=0)

LR_PAIRS = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16),(1,2),(3,4)]

def horizontal_flip_sequence(x: np.ndarray) -> np.ndarray:
    T, F = x.shape
    xy = x.reshape(T, 17, 2).copy()
    xy[..., 0] *= -1.0
    for li, ri in LR_PAIRS:
        tmp = xy[:, li, :].copy()
        xy[:, li, :] = xy[:, ri, :]
        xy[:, ri, :] = tmp
    return xy.reshape(T, 34)

def time_jitter(x: np.ndarray, drop_prob=0.04, dup_prob=0.04, target_len=60) -> np.ndarray:
    out=[]
    for i in range(len(x)):
        if np.random.rand() < drop_prob:   # drop frame
            continue
        out.append(x[i])
        if np.random.rand() < dup_prob and i+1 < len(x):
            out.append(x[i])               # duplicate next
    x = np.array(out, dtype=x.dtype)
    return pad_or_center_trim(x, target_len)

def time_mask(x: np.ndarray, prob=0.15, max_width=8) -> np.ndarray:
    """SpecAugment-style temporal masking: zero-out consecutive frames."""
    if prob <= 0 or max_width <= 0: return x
    T = len(x)
    m = x.copy()
    if np.random.rand() < prob:
        w = np.random.randint(1, max_width+1)
        s = np.random.randint(0, max(1, T - w + 1))
        m[s:s+w] = 0.0
    return m

# =========================
# Dataset
# =========================
class PoseSeqDS(Dataset):
    def __init__(self, entries, seq_len=60, training=False, use_aug=False,
                 time_mask_prob=0.0, time_mask_max=0):
        self.entries = entries
        self.seq_len = seq_len
        self.training = training
        self.use_aug = use_aug
        self.time_mask_prob = time_mask_prob
        self.time_mask_max = time_mask_max

    def __len__(self): return len(self.entries)

    def __getitem__(self, i):
        p, y = self.entries[i]
        x = np.load(p)
        if self.training and self.use_aug:
            x = time_jitter(x, 0.05, 0.05, self.seq_len)
            if random.random() < 0.5:
                x = horizontal_flip_sequence(x)
        else:
            x = pad_or_center_trim(x, self.seq_len)
        if self.training and self.time_mask_prob > 0:
            x = time_mask(x, self.time_mask_prob, self.time_mask_max)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        return x, y

# =========================
# Discover cached data
# =========================
def discover_cache(cache_dir: Path):
    cache_dir = Path(cache_dir)
    index_csv = cache_dir / "index.csv"
    labels_json = cache_dir / "labels.json"

    index = []
    if index_csv.exists():
        import pandas as pd
        df = pd.read_csv(index_csv)
        for p, y in zip(df["seq_path"], df["label_id"]):
            index.append((Path(p), int(y)))
    else:
        for f in sorted(cache_dir.glob("*.npy")):
            m = re.match(r"^(\d+?)__.+?\.npy$", f.name)
            if m:
                index.append((f, int(m.group(1))))

    labels = []
    label2id = {}
    if labels_json.exists():
        with open(labels_json) as f:
            meta = json.load(f)
        labels = meta.get("labels", [])
        label2id = {str(k): int(v) for k,v in meta.get("label2id", {}).items()}
    if not labels:
        max_id = max((y for _,y in index), default=-1)
        labels = [f"class_{i:02d}" for i in range(max_id+1)]
        label2id = {name:i for i,name in enumerate(labels)}
    return index, labels, label2id

def stratified_split(index, val_frac=0.15, test_frac=0.15, seed=42):
    random.seed(seed)
    bylab = defaultdict(list)
    for p,y in index: bylab[y].append((p,y))
    train,val,test = [],[],[]
    for y, L in bylab.items():
        random.shuffle(L)
        n = len(L)
        n_test = max(1, int(round(test_frac*n)))
        n_val  = max(1, int(round(val_frac*n)))
        test  += L[:n_test]
        val   += L[n_test:n_test+n_val]
        train += L[n_test+n_val:]
    return train,val,test

# =========================
# TCN
# =========================
class Chomp1d(nn.Module):
    def __init__(self, c): super().__init__(); self.c=c
    def forward(self, x): return x[:, :, :-self.c].contiguous() if self.c>0 else x

class TemporalBlock(nn.Module):
    def __init__(self, n_in, n_out, k, dilation, padding, dropout=0.2):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv1d(n_in, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
            nn.Conv1d(n_out, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
        )
        self.down = nn.Conv1d(n_in, n_out, 1) if n_in!=n_out else nn.Identity()
    def forward(self, x):
        out = self.seq(x)
        return out + self.down(x)

class TCNClassifier(nn.Module):
    def __init__(self, feats=34, channels=(64,64,128), k=5, dropout=0.2, num_classes=10):
        super().__init__()
        layers=[]; n_in=feats
        for i, ch in enumerate(channels):
            dil = 2**i
            pad = (k-1)*dil
            layers.append(TemporalBlock(n_in, ch, k, dil, pad, dropout))
            n_in = ch
        self.tcn = nn.Sequential(*layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc   = nn.Linear(n_in, num_classes)
    def forward(self, x):              # x: (B,T,F)
        x = x.transpose(1,2)           # (B,F,T)
        y = self.tcn(x)                # (B,C,T)
        y = self.pool(y).squeeze(-1)   # (B,C)
        return self.fc(y)

# =========================
# Loss / sampler
# =========================
def build_loss(use_class_weights, train_entries, num_classes, device, label_smoothing=0.0):
    if use_class_weights:
        counts = Counter([y for _,y in train_entries])
        freq   = np.array([counts.get(i,0) for i in range(num_classes)], dtype=np.float32)
        w = torch.tensor(1.0/np.maximum(freq,1), dtype=torch.float32, device=device)
        w = w / w.mean()
        return nn.CrossEntropyLoss(weight=w, label_smoothing=label_smoothing)
    return nn.CrossEntropyLoss(label_smoothing=label_smoothing)

def build_sampler(use_sampler, train_entries, num_classes):
    if not use_sampler: return None
    counts = Counter([y for _,y in train_entries])
    freq   = np.array([counts.get(i,0) for i in range(num_classes)], dtype=np.float32)
    cw = 1.0/np.maximum(freq,1); cw = cw/ cw.mean()
    weights = np.array([cw[y] for _,y in train_entries], dtype=np.float32)
    return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

# =========================
# Eval helpers
# =========================
@torch.no_grad()
def eval_loader(net, loader, device, criterion):
    net.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = net(xb)
        loss_sum += criterion(logits, yb).item() * yb.size(0)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
    return loss_sum/max(total,1), correct/max(total,1)

@torch.no_grad()
def collect_preds(net, loader, device):
    net.eval()
    preds, trues = [], []
    for xb, yb in loader:
        xb = xb.to(device)
        p = net(xb).argmax(1).cpu().numpy()
        preds.append(p); trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

# =========================
# Pose-correction feedback
# =========================
LHIP,RHIP,LKNEE,RKNEE,LANK,RANK = 11,12,13,14,15,16
LSHO,RSHO,LELB,RELB,LWR,RWR     = 5,6,7,8,9,10

def angle_deg(a, b, c):
    ba = a - b; bc = c - b
    den = (np.linalg.norm(ba)*np.linalg.norm(bc))
    if den == 0: return 180.0
    cosv = np.clip(np.dot(ba,bc)/den, -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))

def angles_from_seq(seq_xy: np.ndarray):
    stats = {"knee_r":[], "knee_l":[], "elbow_r":[], "elbow_l":[]}
    for t in range(len(seq_xy)):
        xy = seq_xy[t].reshape(17,2)
        stats["knee_r"].append(angle_deg(xy[RHIP], xy[RKNEE], xy[RANK]))
        stats["knee_l"].append(angle_deg(xy[LHIP], xy[LKNEE], xy[LANK]))
        stats["elbow_r"].append(angle_deg(xy[RSHO], xy[RELB], xy[RWR]))
        stats["elbow_l"].append(angle_deg(xy[LSHO], xy[LELB], xy[LWR]))
    return {k: {
        "min": float(np.min(v)), "max": float(np.max(v)),
        "p10": float(np.percentile(v,10)), "p90": float(np.percentile(v,90))
    } for k,v in stats.items()}

def feedback_from_stats(label: str, stats: dict):
    label_l = (label or "").lower()
    fb = []
    if any(s in label_l for s in ["squat","leg extension","hip thrust"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min > 100: fb.append("Squat depth shallow — aim for ~90° knee flexion.")
        elif knee_min < 60: fb.append("Very deep — ensure control and comfort range.")
        else: fb.append("Good squat depth.")
    if any(s in label_l for s in ["deadlift","romanian"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min < 70: fb.append("Knees bending a lot — hinge more from hips.")
    if any(s in label_l for s in ["push","press","bench"]):
        emax = max(stats["elbow_l"]["p90"], stats["elbow_r"]["p90"])
        if emax > 130: fb.append("Elbows flared — consider ~45° to protect shoulders.")
        else: fb.append("Elbow tracking looks reasonable.")
    if "curl" in label_l:
        emin = min(stats["elbow_l"]["min"], stats["elbow_r"]["min"])
        if emin > 80: fb.append("Limited elbow flexion — curl through fuller range.")
    if "plank" in label_l:
        fb.append("Keep a straight line head→heels; avoid hip sag.")
    if not fb:
        fb.append("Form looks okay; review depth and control.")
    return fb

@torch.no_grad()
def run_feedback_on_file(npy_path: Path, model, labels, device, seq_len):
    npy_path = Path(npy_path)
    if not npy_path.exists():
        print(f"[analyze_npy] File not found: {npy_path}")
        return
    seq = np.load(npy_path)
    seq = pad_or_center_trim(seq, seq_len)
    x = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(device)
    model.eval()
    logits = model(x)
    pred_id = int(logits.argmax(1).item())
    pred_label = labels[pred_id] if 0 <= pred_id < len(labels) else str(pred_id)
    print(f"[analyze_npy] Predicted: {pred_label} (id={pred_id})")
    stats = angles_from_seq(seq)
    cues = feedback_from_stats(pred_label, stats)
    print("[feedback] angle stats:", json.dumps(stats, indent=2))
    print("[feedback] suggestions:")
    for c in cues: print(" -", c)

# =========================
# MixUp (loss-level, no label smoothing conflict)
# =========================
def mixup_batch(x, y, alpha=0.4):
    if alpha <= 0: return x, y, None
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0), device=x.device)
    x_mix = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return x_mix, (y_a, y_b), lam

def mixup_criterion(criterion, pred, y_mix):
    y_a, y_b, lam = y_mix
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# =========================
# Main
# =========================
def main():
    ap = argparse.ArgumentParser("Train TCN on cached pose sequences (+report, MixUp, time masking)")
    ap.add_argument("--cache_dir", type=Path, required=True)
    ap.add_argument("--seq_len", type=int, default=60)
    ap.add_argument("--batch_size", type=int, default=8)
    ap.add_argument("--epochs", type=int, default=25)
    ap.add_argument("--lr", type=float, default=3e-4)
    ap.add_argument("--weight_decay", type=float, default=1e-4)
    ap.add_argument("--channels", type=str, default="64,64,128")
    ap.add_argument("--kernel", type=int, default=5)
    ap.add_argument("--dropout", type=float, default=0.2)
    ap.add_argument("--use_aug", action="store_true")
    ap.add_argument("--use_class_weights", action="store_true")
    ap.add_argument("--use_sampler", action="store_true")
    ap.add_argument("--label_smoothing", type=float, default=0.0)
    ap.add_argument("--scheduler", type=str, default="cosine", choices=["none","cosine"])
    ap.add_argument("--patience", type=int, default=6)
    ap.add_argument("--save_path", type=Path, default=Path("./best_tcn.pt"))
    ap.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--eval_only", action="store_true")
    ap.add_argument("--checkpoint", type=Path, default=None)
    ap.add_argument("--analyze_npy", type=Path, default=None)

    # New flags
    ap.add_argument("--report", action="store_true", help="print per-class report & confusion matrix")
    ap.add_argument("--report_path", type=Path, default=None, help="optional CSV save dir for report/matrix")
    ap.add_argument("--mixup", action="store_true")
    ap.add_argument("--mixup_alpha", type=float, default=0.4)
    ap.add_argument("--time_mask_prob", type=float, default=0.0)
    ap.add_argument("--time_mask_max", type=int, default=0)
    args = ap.parse_args()

    set_seed(args.seed)

    # Data
    index, labels, label2id = discover_cache(args.cache_dir)
    num_classes = len(labels)
    if len(index) == 0:
        raise SystemExit(f"No sequences found in {args.cache_dir}")

    train, val, test = stratified_split(index, val_frac=0.15, test_frac=0.15, seed=args.seed)

    pin = torch.cuda.is_available()
    sampler = build_sampler(args.use_sampler, train, num_classes)
    train_ds = PoseSeqDS(train, args.seq_len, True, args.use_aug, args.time_mask_prob, args.time_mask_max)
    val_ds   = PoseSeqDS(val,   args.seq_len, False, False, 0.0, 0)
    test_ds  = PoseSeqDS(test,  args.seq_len, False, False, 0.0, 0)

    train_dl = DataLoader(train_ds, batch_size=args.batch_size,
                          shuffle=(sampler is None), sampler=sampler,
                          num_workers=2, pin_memory=pin)
    val_dl   = DataLoader(val_ds,   batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=pin)
    test_dl  = DataLoader(test_ds,  batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=pin)

    # Model & training utils
    channels = tuple(int(x) for x in args.channels.split(",") if x.strip())
    model = TCNClassifier(feats=34, channels=channels, k=args.kernel, dropout=args.dropout,
                          num_classes=num_classes).to(args.device)
    criterion = build_loss(args.use_class_weights, train, num_classes, args.device, args.label_smoothing)
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs) if args.scheduler=="cosine" else None

    if args.checkpoint and Path(args.checkpoint).exists():
        model.load_state_dict(torch.load(args.checkpoint, map_location=args.device))
        print(f"Loaded checkpoint: {args.checkpoint}")

    # Eval-only path
    if args.eval_only:
        vl, va = eval_loader(model, val_dl, args.device, criterion)
        tl, ta = eval_loader(model, test_dl, args.device, criterion)
        print(f"[EvalOnly] val_loss={vl:.4f} val_acc={va:.3f} | test_loss={tl:.4f} test_acc={ta:.3f}")
        if args.report:
            do_report(model, val_dl, test_dl, labels, args.device, args.report_path)
        if args.analyze_npy:
            run_feedback_on_file(args.analyze_npy, model, labels, args.device, args.seq_len)
        return

    # Train
    best_val, bad = float("inf"), 0
    for ep in range(1, args.epochs+1):
        model.train(); running=0.0; batches=0
        for xb, yb in train_dl:
            xb, yb = xb.to(args.device), yb.to(args.device)
            # MixUp (loss-level)
            if args.mixup:
                x_mix, y_mix, lam = mixup_batch(xb, yb, alpha=args.mixup_alpha)
                logits = model(x_mix)
                loss = mixup_criterion(criterion, logits, (y_mix[0], y_mix[1], lam))
            else:
                logits = model(xb)
                loss = criterion(logits, yb)
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            running += loss.item(); batches += 1

        tr = running/max(batches,1)
        vl, va = eval_loader(model, val_dl, args.device, criterion)
        if scheduler: scheduler.step()
        print(f"Epoch {ep:02d} | train={tr:.4f} | val_loss={vl:.4f} | val_acc={va:.3f}")

        if vl < best_val:
            best_val, bad = vl, 0
            torch.save(model.state_dict(), args.save_path)
        else:
            bad += 1
            if bad >= args.patience:
                print("⏹️ Early stopping."); break

    # Test best
    model.load_state_dict(torch.load(args.save_path, map_location=args.device))
    tl, ta = eval_loader(model, test_dl, args.device, criterion)
    print(f"🏁 Test(best): loss={tl:.4f} | acc={ta:.3f}")

    if args.report:
        do_report(model, val_dl, test_dl, labels, args.device, args.report_path)

    if args.analyze_npy:
        run_feedback_on_file(args.analyze_npy, model, labels, args.device, args.seq_len)

# =========================
# Reporting
# =========================
def do_report(model, val_dl, test_dl, labels, device, out_dir=None):
    try:
        from sklearn.metrics import classification_report, confusion_matrix
    except Exception:
        print("scikit-learn not available; installing...")
        import subprocess, sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scikit-learn"])
        from sklearn.metrics import classification_report, confusion_matrix

    pv, yv = collect_preds(model, val_dl, device)
    pt, yt = collect_preds(model, test_dl, device)

    print("\n=== Validation Report ===")
    print(classification_report(yv, pv, target_names=labels, digits=3))
    print(confusion_matrix(yv, pv))

    print("\n=== Test Report ===")
    print(classification_report(yt, pt, target_names=labels, digits=3))
    cm = confusion_matrix(yt, pt)
    print(cm)

    if out_dir:
        out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
        import pandas as pd
        # Save classification reports as CSV (per-split)
        def report_to_df(y_true, y_pred):
            from sklearn.metrics import precision_recall_fscore_support
            p,r,f,s = precision_recall_fscore_support(y_true, y_pred, labels=list(range(len(labels))), zero_division=0)
            return pd.DataFrame({"label": labels, "precision": p, "recall": r, "f1": f, "support": s})
        report_to_df(yv, pv).to_csv(out_dir/"val_report.csv", index=False)
        report_to_df(yt, pt).to_csv(out_dir/"test_report.csv", index=False)
        pd.DataFrame(cm, index=labels, columns=labels).to_csv(out_dir/"test_confusion_matrix.csv", index=True)
        print(f"📁 Reports saved to: {out_dir}")

if __name__ == "__main__":
    main()

Overwriting train_tcn.py


In [ ]:
!python train_tcn.py --cache_dir /content/pose_seq_yolo \
  --epochs 25 --use_aug --use_class_weights \
  --mixup --mixup_alpha 0.4 \
  --time_mask_prob 0.15 --time_mask_max 8 \
  --report --save_path /content/best_tcn.pt

Epoch 01 | train=3.0591 | val_loss=2.8526 | val_acc=0.242
Epoch 02 | train=2.7724 | val_loss=2.4993 | val_acc=0.316
Epoch 03 | train=2.4539 | val_loss=2.2049 | val_acc=0.368
Epoch 04 | train=2.2369 | val_loss=2.0637 | val_acc=0.337
Epoch 05 | train=2.2111 | val_loss=1.9934 | val_acc=0.400
Epoch 06 | train=2.0564 | val_loss=1.8079 | val_acc=0.474
Epoch 07 | train=1.9789 | val_loss=1.7255 | val_acc=0.400
Epoch 08 | train=1.8966 | val_loss=1.9230 | val_acc=0.379
Epoch 09 | train=1.9426 | val_loss=1.7095 | val_acc=0.442
Epoch 10 | train=1.7579 | val_loss=1.6287 | val_acc=0.484
Epoch 11 | train=1.8859 | val_loss=1.5490 | val_acc=0.547
Epoch 12 | train=1.8561 | val_loss=1.5859 | val_acc=0.474
Epoch 13 | train=1.7326 | val_loss=1.5802 | val_acc=0.526
Epoch 14 | train=1.7660 | val_loss=1.5684 | val_acc=0.474
Epoch 15 | train=1.7377 | val_loss=1.4501 | val_acc=0.547
Epoch 16 | train=1.6827 | val_loss=1.4484 | val_acc=0.526
Epoch 17 | train=1.6140 | val_loss=1.4054 | val_acc=0.558
Epoch 18 | tra

In [ ]:
!python train_tcn.py --cache_dir /content/pose_seq_yolo \
  --epochs 35 --batch_size 8 \
  --use_aug --use_class_weights --use_sampler \
  --mixup --mixup_alpha 0.3 \
  --time_mask_prob 0.10 --time_mask_max 12 \
  --label_smoothing 0.05 \
  --scheduler cosine --patience 10 \
  --seq_len 90 --channels 64,128,256 --kernel 7 \
  --report --report_path /content/tcn_reports \
  --save_path /content/best_tcn.pt

Epoch 01 | train=2.7115 | val_loss=3.3627 | val_acc=0.105
Epoch 02 | train=2.2331 | val_loss=2.7081 | val_acc=0.242
Epoch 03 | train=2.1067 | val_loss=2.5825 | val_acc=0.168
Epoch 04 | train=1.8979 | val_loss=2.5084 | val_acc=0.232
Epoch 05 | train=1.8633 | val_loss=2.5581 | val_acc=0.189
Epoch 06 | train=1.7276 | val_loss=2.3701 | val_acc=0.274
Epoch 07 | train=1.8104 | val_loss=2.2485 | val_acc=0.242
Epoch 08 | train=1.5089 | val_loss=2.2910 | val_acc=0.316
Epoch 09 | train=1.5474 | val_loss=2.2000 | val_acc=0.316
Epoch 10 | train=1.5363 | val_loss=1.9979 | val_acc=0.421
Epoch 11 | train=1.5638 | val_loss=2.1056 | val_acc=0.337
Epoch 12 | train=1.5359 | val_loss=2.0524 | val_acc=0.400
Epoch 13 | train=1.5344 | val_loss=2.0856 | val_acc=0.389
Epoch 14 | train=1.5156 | val_loss=1.9244 | val_acc=0.474
Epoch 15 | train=1.4428 | val_loss=1.9205 | val_acc=0.421
Epoch 16 | train=1.3795 | val_loss=1.9190 | val_acc=0.484
Epoch 17 | train=1.4324 | val_loss=1.9176 | val_acc=0.432
Epoch 18 | tra

In [ ]:
!python train_tcn.py --cache_dir /content/pose_seq_yolo \
  --epochs 30 --batch_size 8 \
  --use_aug --use_class_weights --use_sampler \
  --mixup --mixup_alpha 0.4 \
  --time_mask_prob 0.15 --time_mask_max 8 \
  --scheduler cosine --patience 8 \
  --report --report_path /content/tcn_reports \
  --save_path /content/best_tcn.pt

Epoch 01 | train=2.9542 | val_loss=3.0629 | val_acc=0.042
Epoch 02 | train=2.5669 | val_loss=2.7816 | val_acc=0.126
Epoch 03 | train=2.2166 | val_loss=2.5918 | val_acc=0.179
Epoch 04 | train=1.9424 | val_loss=2.3237 | val_acc=0.232
Epoch 05 | train=1.9853 | val_loss=2.1394 | val_acc=0.284
Epoch 06 | train=1.8343 | val_loss=1.9797 | val_acc=0.316
Epoch 07 | train=1.7205 | val_loss=1.9446 | val_acc=0.274
Epoch 08 | train=1.6631 | val_loss=1.8821 | val_acc=0.263
Epoch 09 | train=1.6619 | val_loss=1.9026 | val_acc=0.295
Epoch 10 | train=1.5073 | val_loss=1.8037 | val_acc=0.305
Epoch 11 | train=1.6444 | val_loss=1.9205 | val_acc=0.316
Epoch 12 | train=1.6016 | val_loss=1.8508 | val_acc=0.253
Epoch 13 | train=1.4852 | val_loss=1.9791 | val_acc=0.326
Epoch 14 | train=1.5401 | val_loss=1.8900 | val_acc=0.326
Epoch 15 | train=1.5380 | val_loss=1.7546 | val_acc=0.347
Epoch 16 | train=1.4474 | val_loss=1.7630 | val_acc=0.295
Epoch 17 | train=1.3124 | val_loss=1.8281 | val_acc=0.347
Epoch 18 | tra

In [ ]:
!python train_tcn.py --cache_dir /content/pose_seq_yolo \
  --eval_only --checkpoint /content/best_tcn.pt \
  --channels 64,128,256 --kernel 7 \
  --analyze_npy "/content/pose_seq_yolo/14__push-up_36.npy"

Loaded checkpoint: /content/best_tcn.pt
[EvalOnly] val_loss=1.5224 val_acc=0.526 | test_loss=1.3350 test_acc=0.526
[analyze_npy] Predicted: push-up (id=14)
[feedback] angle stats: {
  "knee_r": {
    "min": 0.8965660929679871,
    "max": 148.65103149414062,
    "p10": 18.443626308441164,
    "p90": 142.558251953125
  },
  "knee_l": {
    "min": 121.40056610107422,
    "max": 161.4187774658203,
    "p10": 123.7725212097168,
    "p90": 153.2855712890625
  },
  "elbow_r": {
    "min": 55.78034210205078,
    "max": 169.14776611328125,
    "p10": 87.44112701416016,
    "p90": 166.6210693359375
  },
  "elbow_l": {
    "min": 68.80013275146484,
    "max": 146.9849090576172,
    "p10": 73.56137542724609,
    "p90": 142.16739807128906
  }
}
[feedback] suggestions:
 - Elbows flared — consider ~45° to protect shoulders.


In [ ]:
%%writefile train_tcn.py

#!/usr/bin/env python3
import argparse, json, random, re
from pathlib import Path
from collections import defaultdict, Counter

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# ------------------------- Repro -------------------------
def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# -------------------- Sequence utilities ----------------
def pad_or_center_trim(x: np.ndarray, T: int) -> np.ndarray:
    t = len(x)
    if t == T: return x
    if t > T:
        s = (t - T)//2
        return x[s:s+T]
    pad = np.zeros((T - t, x.shape[1]), dtype=x.dtype)
    return np.concatenate([x, pad], axis=0)

LR_PAIRS = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16),(1,2),(3,4)]

def horizontal_flip_sequence(x: np.ndarray) -> np.ndarray:
    T, F = x.shape
    xy = x.reshape(T, 17, 2).copy()
    xy[..., 0] *= -1.0
    for li, ri in LR_PAIRS:
        tmp = xy[:, li, :].copy()
        xy[:, li, :] = xy[:, ri, :]
        xy[:, ri, :] = tmp
    return xy.reshape(T, 34)

def time_jitter(x: np.ndarray, drop_prob=0.04, dup_prob=0.04, target_len=60) -> np.ndarray:
    out=[]
    for i in range(len(x)):
        if np.random.rand() < drop_prob:   # drop
            continue
        out.append(x[i])
        if np.random.rand() < dup_prob and i+1 < len(x):
            out.append(x[i])               # dup
    x = np.array(out, dtype=x.dtype)
    return pad_or_center_trim(x, target_len)

def time_mask(x: np.ndarray, prob=0.0, max_width=0) -> np.ndarray:
    if prob <= 0 or max_width <= 0: return x
    T = len(x)
    m = x.copy()
    if np.random.rand() < prob:
        w = np.random.randint(1, max_width+1)
        s = np.random.randint(0, max(1, T - w + 1))
        m[s:s+w] = 0.0
    return m

# -------------------------- Dataset ----------------------
class PoseSeqDS(Dataset):
    def __init__(self, entries, seq_len=60, training=False, use_aug=False,
                 time_mask_prob=0.0, time_mask_max=0):
        self.entries = entries
        self.seq_len = seq_len
        self.training = training
        self.use_aug = use_aug
        self.time_mask_prob = time_mask_prob
        self.time_mask_max = time_mask_max

    def __len__(self): return len(self.entries)

    def __getitem__(self, i):
        p, y = self.entries[i]
        x = np.load(p)
        if self.training and self.use_aug:
            x = time_jitter(x, 0.05, 0.05, self.seq_len)
            if random.random() < 0.5:
                x = horizontal_flip_sequence(x)
        else:
            x = pad_or_center_trim(x, self.seq_len)
        if self.training and self.time_mask_prob > 0:
            x = time_mask(x, self.time_mask_prob, self.time_mask_max)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        return x, y

# --------------------- Data discovery/split ---------------
def discover_cache(cache_dir: Path):
    cache_dir = Path(cache_dir)
    index_csv = cache_dir / "index.csv"
    labels_json = cache_dir / "labels.json"

    index = []
    if index_csv.exists():
        import pandas as pd
        df = pd.read_csv(index_csv)
        for p, y in zip(df["seq_path"], df["label_id"]):
            index.append((Path(p), int(y)))
    else:
        for f in sorted(cache_dir.glob("*.npy")):
            m = re.match(r"^(\d+?)__.+?\.npy$", f.name)
            if m:
                index.append((f, int(m.group(1))))

    labels, label2id = [], {}
    if labels_json.exists():
        with open(labels_json) as f:
            meta = json.load(f)
        labels = meta.get("labels", [])
        label2id = {str(k): int(v) for k,v in meta.get("label2id", {}).items()}
    if not labels:
        max_id = max((y for _,y in index), default=-1)
        labels = [f"class_{i:02d}" for i in range(max_id+1)]
        label2id = {name:i for i,name in enumerate(labels)}
    return index, labels, label2id

def stratified_split(index, val_frac=0.15, test_frac=0.15, seed=42):
    random.seed(seed)
    bylab = defaultdict(list)
    for p,y in index: bylab[y].append((p,y))
    train,val,test = [],[],[]
    for y, L in bylab.items():
        random.shuffle(L)
        n = len(L)
        n_test = max(1, int(round(test_frac*n)))
        n_val  = max(1, int(round(val_frac*n)))
        test  += L[:n_test]
        val   += L[n_test:n_test+n_val]
        train += L[n_test+n_val:]
    return train,val,test

# --------------------------- TCN --------------------------
class Chomp1d(nn.Module):
    def __init__(self, c): super().__init__(); self.c=c
    def forward(self, x): return x[:, :, :-self.c].contiguous() if self.c>0 else x

class TemporalBlock(nn.Module):
    def __init__(self, n_in, n_out, k, dilation, padding, dropout=0.2):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv1d(n_in, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
            nn.Conv1d(n_out, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
        )
        self.down = nn.Conv1d(n_in, n_out, 1) if n_in!=n_out else nn.Identity()
    def forward(self, x):
        out = self.seq(x)
        return out + self.down(x)

class TCNClassifier(nn.Module):
    def __init__(self, feats=34, channels=(64,64,128), k=5, dropout=0.2, num_classes=10):
        super().__init__()
        layers=[]; n_in=feats
        for i, ch in enumerate(channels):
            dil = 2**i
            pad = (k-1)*dil
            layers.append(TemporalBlock(n_in, ch, k, dil, pad, dropout))
            n_in = ch
        self.tcn = nn.Sequential(*layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc   = nn.Linear(n_in, num_classes)
    def forward(self, x):              # x: (B,T,F)
        x = x.transpose(1,2)           # (B,F,T)
        y = self.tcn(x)                # (B,C,T)
        y = self.pool(y).squeeze(-1)   # (B,C)
        return self.fc(y)

# -------------------- Loss / sampler / eval ---------------
def build_loss(use_class_weights, train_entries, num_classes, device, label_smoothing=0.0):
    if use_class_weights:
        counts = Counter([y for _,y in train_entries])
        freq   = np.array([counts.get(i,0) for i in range(num_classes)], dtype=np.float32)
        w = torch.tensor(1.0/np.maximum(freq,1), dtype=torch.float32, device=device)
        w = w / w.mean()
        return nn.CrossEntropyLoss(weight=w, label_smoothing=label_smoothing, reduction='none')
    return nn.CrossEntropyLoss(label_smoothing=label_smoothing, reduction='none')

def build_sampler(use_sampler, train_entries, num_classes):
    if not use_sampler: return None
    counts = Counter([y for _,y in train_entries])
    freq   = np.array([counts.get(i,0) for i in range(num_classes)], dtype=np.float32)
    cw = 1.0/np.maximum(freq,1); cw = cw/ cw.mean()
    weights = np.array([cw[y] for _,y in train_entries], dtype=np.float32)
    return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

@torch.no_grad()
def eval_loader(net, loader, device, criterion_reduced):
    net.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = net(xb)
        loss = criterion_reduced(logits, yb)
        loss_sum += float(loss) * yb.size(0)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        total += yb.size(0)
    return loss_sum/max(total,1), correct/max(total,1)

def reduce_mean(loss_vec):
    # loss_vec is (B,), already per-sample (because we set reduction='none')
    return loss_vec.mean()

# ------------------------ TTA helpers ---------------------
def predict_with_tta(model, xb, device):
    # xb: (B,T,34)
    logits = model(xb.to(device))
    x_np = xb.cpu().numpy()
    x_flip_np = np.stack([horizontal_flip_sequence(xx) for xx in x_np], axis=0)
    x_flip = torch.tensor(x_flip_np, dtype=xb.dtype, device=device)
    logits_flip = model(x_flip)
    return (logits + logits_flip) / 2

@torch.no_grad()
def collect_preds(model, loader, device, use_tta=False):
    model.eval()
    preds, trues = [], []
    for xb, yb in loader:
        if use_tta:
            logits = predict_with_tta(model, xb, device)
        else:
            logits = model(xb.to(device))
        p = logits.argmax(1).cpu().numpy()
        preds.append(p); trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

# ---------------- Pose-correction feedback (angles) -------
LHIP,RHIP,LKNEE,RKNEE,LANK,RANK = 11,12,13,14,15,16
LSHO,RSHO,LELB,RELB,LWR,RWR     = 5,6,7,8,9,10

def angle_deg(a, b, c):
    ba = a - b; bc = c - b
    den = (np.linalg.norm(ba)*np.linalg.norm(bc))
    if den == 0: return 180.0
    cosv = np.clip(np.dot(ba,bc)/den, -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))

def angles_from_seq(seq_xy: np.ndarray):
    stats = {"knee_r":[], "knee_l":[], "elbow_r":[], "elbow_l":[]}
    for t in range(len(seq_xy)):
        xy = seq_xy[t].reshape(17,2)
        stats["knee_r"].append(angle_deg(xy[RHIP], xy[RKNEE], xy[RANK]))
        stats["knee_l"].append(angle_deg(xy[LHIP], xy[LKNEE], xy[LANK]))
        stats["elbow_r"].append(angle_deg(xy[RSHO], xy[RELB], xy[RWR]))
        stats["elbow_l"].append(angle_deg(xy[LSHO], xy[LELB], xy[LWR]))
    return {k: {
        "min": float(np.min(v)), "max": float(np.max(v)),
        "p10": float(np.percentile(v,10)), "p90": float(np.percentile(v,90))
    } for k,v in stats.items()}

def feedback_from_stats(label: str, stats: dict):
    label_l = (label or "").lower()
    fb = []
    if any(s in label_l for s in ["squat","leg extension","hip thrust"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min > 100: fb.append("Squat depth shallow — aim for ~90° knee flexion.")
        elif knee_min < 60: fb.append("Very deep — ensure control and comfort range.")
        else: fb.append("Good squat depth.")
    if any(s in label_l for s in ["deadlift","romanian"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min < 70: fb.append("Knees bending a lot — hinge more from hips.")
    if any(s in label_l for s in ["push","press","bench"]):
        emax = max(stats["elbow_l"]["p90"], stats["elbow_r"]["p90"])
        if emax > 130: fb.append("Elbows flared — consider ~45° to protect shoulders.")
        else: fb.append("Elbow tracking looks reasonable.")
    if "curl" in label_l:
        emin = min(stats["elbow_l"]["min"], stats["elbow_r"]["min"])
        if emin > 80: fb.append("Limited elbow flexion — curl through fuller range.")
    if "plank" in label_l:
        fb.append("Keep a straight line head→heels; avoid hip sag.")
    if not fb:
        fb.append("Form looks okay; review depth and control.")
    return fb

@torch.no_grad()
def run_feedback_on_file(npy_path: Path, model, labels, device, seq_len):
    npy_path = Path(npy_path)
    if not npy_path.exists():
        print(f"[analyze_npy] File not found: {npy_path}")
        return
    seq = np.load(npy_path)
    seq = pad_or_center_trim(seq, seq_len)
    x = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).to(device)
    model.eval()
    logits = model(x)
    pred_id = int(logits.argmax(1).item())
    pred_label = labels[pred_id] if 0 <= pred_id < len(labels) else str(pred_id)
    print(f"[analyze_npy] Predicted: {pred_label} (id={pred_id})")
    stats = angles_from_seq(seq)
    cues = feedback_from_stats(pred_label, stats)
    print("[feedback] angle stats:", json.dumps(stats, indent=2))
    print("[feedback] suggestions:")
    for c in cues: print(" -", c)

# ------------------------- MixUp / Focal ------------------
def mixup_batch(x, y, alpha=0.4):
    if alpha <= 0: return x, y, None
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0), device=x.device)
    x_mix = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return x_mix, (y_a, y_b), lam

def mixup_criterion(criterion_vec, pred, y_mix):
    # criterion_vec returns (B,) per-sample; we need mean after weighting
    y_a, y_b, lam = y_mix
    loss_a = criterion_vec(pred, y_a)
    loss_b = criterion_vec(pred, y_b)
    return (lam * loss_a + (1 - lam) * loss_b).mean()

class FocalWrapper(nn.Module):
    """Wraps a per-sample CE (reduction='none') with focal modulation."""
    def __init__(self, base_ce, gamma=1.5):
        super().__init__(); self.ce=base_ce; self.gamma=gamma
    def forward(self, logits, target):
        ce = self.ce(logits, target)                      # (B,)
        with torch.no_grad():
            pt = torch.softmax(logits, dim=1).gather(1, target.unsqueeze(1)).clamp_min(1e-8).squeeze(1)
        mod = (1 - pt) ** self.gamma                      # (B,)
        return (mod * ce)                                 # keep per-sample; caller should mean()

# --------------------------- Report -----------------------
def do_report(model, val_dl, test_dl, labels, device, out_dir=None, use_tta=False):
    try:
        from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
    except Exception:
        import subprocess, sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scikit-learn"])
        from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

    pv, yv = collect_preds(model, val_dl, device, use_tta)
    pt, yt = collect_preds(model, test_dl, device, use_tta)

    print("\n=== Validation Report ===")
    print(classification_report(yv, pv, target_names=labels, digits=3))
    print(confusion_matrix(yv, pv))

    print("\n=== Test Report ===")
    print(classification_report(yt, pt, target_names=labels, digits=3))
    cm = confusion_matrix(yt, pt)
    print(cm)

    if out_dir:
        out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
        import pandas as pd
        p,r,f,s = precision_recall_fscore_support(yt, pt, labels=list(range(len(labels))), zero_division=0)
        pd.DataFrame({"label":labels,"precision":p,"recall":r,"f1":f,"support":s}).to_csv(out_dir/"test_report.csv", index=False)
        pd.DataFrame(cm, index=labels, columns=labels).to_csv(out_dir/"test_confusion_matrix.csv", index=True)
        print(f"📁 Reports saved to: {out_dir}")

# ---------------------------- Main ------------------------
def main():
    ap = argparse.ArgumentParser("Train TCN on cached pose sequences (+TTA, focal, hparam-aware ckpt)")
    ap.add_argument("--cache_dir", type=Path, required=True)
    ap.add_argument("--seq_len", type=int, default=60)
    ap.add_argument("--batch_size", type=int, default=8)
    ap.add_argument("--epochs", type=int, default=25)
    ap.add_argument("--lr", type=float, default=3e-4)
    ap.add_argument("--weight_decay", type=float, default=1e-4)
    ap.add_argument("--channels", type=str, default="64,64,128")
    ap.add_argument("--kernel", type=int, default=5)
    ap.add_argument("--dropout", type=float, default=0.2)
    ap.add_argument("--use_aug", action="store_true")
    ap.add_argument("--use_class_weights", action="store_true")
    ap.add_argument("--use_sampler", action="store_true")
    ap.add_argument("--label_smoothing", type=float, default=0.0)
    ap.add_argument("--scheduler", type=str, default="cosine", choices=["none","cosine"])
    ap.add_argument("--patience", type=int, default=6)
    ap.add_argument("--save_path", type=Path, default=Path("./best_tcn.pt"))
    ap.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--eval_only", action="store_true")
    ap.add_argument("--checkpoint", type=Path, default=None)
    ap.add_argument("--analyze_npy", type=Path, default=None)

    # NEW:
    ap.add_argument("--report", action="store_true")
    ap.add_argument("--report_path", type=Path, default=None)
    ap.add_argument("--tta", action="store_true", help="horizontal flip TTA at eval/report")
    ap.add_argument("--mixup", action="store_true")
    ap.add_argument("--mixup_alpha", type=float, default=0.4)
    ap.add_argument("--time_mask_prob", type=float, default=0.0)
    ap.add_argument("--time_mask_max", type=int, default=0)
    ap.add_argument("--focal", action="store_true")
    ap.add_argument("--focal_gamma", type=float, default=1.5)
    args = ap.parse_args()

    set_seed(args.seed)

    # Data
    index, labels, label2id = discover_cache(args.cache_dir)
    num_classes = len(labels)
    if len(index) == 0:
        raise SystemExit(f"No sequences found in {args.cache_dir}")

    train, val, test = stratified_split(index, val_frac=0.15, test_frac=0.15, seed=args.seed)

    pin = torch.cuda.is_available()
    sampler = build_sampler(args.use_sampler, train, num_classes)
    train_ds = PoseSeqDS(train, args.seq_len, True, args.use_aug, args.time_mask_prob, args.time_mask_max)
    val_ds   = PoseSeqDS(val,   args.seq_len, False, False, 0.0, 0)
    test_ds  = PoseSeqDS(test,  args.seq_len, False, False, 0.0, 0)

    train_dl = DataLoader(train_ds, batch_size=args.batch_size,
                          shuffle=(sampler is None), sampler=sampler,
                          num_workers=2, pin_memory=pin)
    val_dl   = DataLoader(val_ds,   batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=pin)
    test_dl  = DataLoader(test_ds,  batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=pin)

    # Model & training utils
    channels = tuple(int(x) for x in args.channels.split(",") if x.strip())
    model = TCNClassifier(feats=34, channels=channels, k=args.kernel, dropout=args.dropout,
                          num_classes=num_classes).to(args.device)

    # Per-sample CE (reduction='none'); we mean it manually to support Focal/MixUp
    base_ce = build_loss(args.use_class_weights, train, num_classes, args.device, args.label_smoothing)
    criterion_vec = FocalWrapper(base_ce, gamma=args.focal_gamma) if args.focal else base_ce
    criterion_reduced = lambda pred, target: reduce_mean(criterion_vec(pred, target))

    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs) if args.scheduler=="cosine" else None

    # --------- Load checkpoint (hparam-aware) ----------
    if args.checkpoint and Path(args.checkpoint).exists():
        ckpt_obj = torch.load(args.checkpoint, map_location=args.device)
        if isinstance(ckpt_obj, dict) and "state_dict" in ckpt_obj:
            hp = ckpt_obj.get("hparams", {})
            ch_from_ckpt = tuple(hp.get("channels", channels))
            k_from_ckpt  = int(hp.get("kernel", args.kernel))
            if ch_from_ckpt != channels or k_from_ckpt != args.kernel:
                print(f"[info] Overriding model hparams from checkpoint: channels={ch_from_ckpt}, kernel={k_from_ckpt}")
                channels = ch_from_ckpt
                args.kernel = k_from_ckpt
                model = TCNClassifier(feats=34, channels=channels, k=args.kernel,
                                      dropout=args.dropout, num_classes=num_classes).to(args.device)
            model.load_state_dict(ckpt_obj["state_dict"], strict=True)
            if "labels" in ckpt_obj and ckpt_obj["labels"]:
                labels = ckpt_obj["labels"]; num_classes = len(labels)
            print(f"Loaded checkpoint: {args.checkpoint}")
        else:
            model.load_state_dict(ckpt_obj, strict=True)
            print(f"Loaded raw state_dict: {args.checkpoint}")

    # --------- Eval-only? ----------
    if args.eval_only:
        vl, va = eval_loader(model, val_dl, args.device, criterion_reduced)
        tl, ta = eval_loader(model, test_dl, args.device, criterion_reduced)
        print(f"[EvalOnly] val_loss={vl:.4f} val_acc={va:.3f} | test_loss={tl:.4f} test_acc={ta:.3f}")
        if args.report:
            do_report(model, val_dl, test_dl, labels, args.device, out_dir=args.report_path, use_tta=args.tta)
        if args.analyze_npy:
            run_feedback_on_file(args.analyze_npy, model, labels, args.device, args.seq_len)
        return

    # ----------------- Train -----------------
    best_val, bad = float("inf"), 0
    for ep in range(1, args.epochs+1):
        model.train(); running=0.0; batches=0
        for xb, yb in train_dl:
            xb, yb = xb.to(args.device), yb.to(args.device)
            if args.mixup:
                x_mix, y_mix, lam = mixup_batch(xb, yb, alpha=args.mixup_alpha)
                logits = model(x_mix)
                loss = mixup_criterion(criterion_vec, logits, (y_mix[0], y_mix[1], lam))
            else:
                logits = model(xb)
                loss = reduce_mean(criterion_vec(logits, yb))
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            running += float(loss); batches += 1

        tr = running/max(batches,1)
        vl, va = eval_loader(model, val_dl, args.device, criterion_reduced)
        if scheduler: scheduler.step()
        print(f"Epoch {ep:02d} | train={tr:.4f} | val_loss={vl:.4f} | val_acc={va:.3f}")

        if vl < best_val:
            best_val, bad = vl, 0
            ckpt = {
                "state_dict": model.state_dict(),
                "hparams": {
                    "feats": 34,
                    "channels": channels,
                    "kernel": args.kernel,
                    "dropout": args.dropout,
                    "num_classes": num_classes,
                    "seq_len": args.seq_len
                },
                "labels": labels
            }
            torch.save(ckpt, args.save_path)
        else:
            bad += 1
            if bad >= args.patience:
                print("⏹️ Early stopping."); break

    # ----------------- Test best -----------------
    ckpt_obj = torch.load(args.save_path, map_location=args.device)
    if isinstance(ckpt_obj, dict) and "state_dict" in ckpt_obj:
        hp = ckpt_obj.get("hparams", {})
        ch_from_ckpt = tuple(hp.get("channels", channels))
        k_from_ckpt  = int(hp.get("kernel", args.kernel))
        if ch_from_ckpt != channels or k_from_ckpt != args.kernel:
            print(f"[info] Using saved hparams for best model: channels={ch_from_ckpt}, kernel={k_from_ckpt}")
            channels = ch_from_ckpt
            args.kernel = k_from_ckpt
            model = TCNClassifier(feats=34, channels=channels, k=args.kernel,
                                  dropout=args.dropout, num_classes=num_classes).to(args.device)
        model.load_state_dict(ckpt_obj["state_dict"], strict=True)
        if "labels" in ckpt_obj and ckpt_obj["labels"]:
            labels = ckpt_obj["labels"]; num_classes = len(labels)
    else:
        model.load_state_dict(ckpt_obj, strict=True)

    tl, ta = eval_loader(model, test_dl, args.device, criterion_reduced)
    print(f"🏁 Test(best): loss={tl:.4f} | acc={ta:.3f}")

    if args.report:
        do_report(model, val_dl, test_dl, labels, args.device, out_dir=args.report_path, use_tta=args.tta)

    if args.analyze_npy:
        run_feedback_on_file(args.analyze_npy, model, labels, args.device, args.seq_len)

if __name__ == "__main__":
    main()

Overwriting train_tcn.py


In [ ]:
!python train_tcn.py --cache_dir /content/pose_seq_yolo \
  --epochs 35 --batch_size 8 \
  --use_aug --use_class_weights --use_sampler \
  --mixup --mixup_alpha 0.3 \
  --time_mask_prob 0.10 --time_mask_max 12 \
  --label_smoothing 0.05 --focal --focal_gamma 1.5 \
  --scheduler cosine --patience 10 \
  --seq_len 90 --channels 64,128,256 --kernel 7 \
  --report --tta --report_path /content/tcn_reports_v3 \
  --save_path /content/best_tcn.pt

/content/train_tcn.py:471: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  running += float(loss); batches += 1
Epoch 01 | train=2.4562 | val_loss=1.9691 | val_acc=0.105
Epoch 02 | train=1.8370 | val_loss=1.5205 | val_acc=0.253
Epoch 03 | train=1.7210 | val_loss=1.4516 | val_acc=0.179
Epoch 04 | train=1.4384 | val_loss=1.3334 | val_acc=0.232
Epoch 05 | train=1.4185 | val_loss=1.3118 | val_acc=0.158
Epoch 06 | train=1.2706 | val_loss=1.1804 | val_acc=0.284
Epoch 07 | train=1.2926 | val_loss=1.1497 | val_acc=0.295
Epoch 08 | train=1.1188 | val_loss=1.1371 | val_acc=0.337
Epoch 09 | train=1.1350 | val_loss=1.0501 | val_acc=0.305
Epoch 10 | train=1.1366 | val_loss=0.9246 | val_acc=0.389
Epoch 11 | train=1.1517 | val_loss=1.0527 | val_acc=0.379
Epoch 12 | train=1.0911 | val_loss=0.8916 | val_acc=0.411


In [ ]:
%%writefile infer_baseline.py
#!/usr/bin/env python3
# infer_baseline.py
import argparse, json, re
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn

# ------------------- TCN (baseline) -------------------
class Chomp1d(nn.Module):
    def __init__(self, c): super().__init__(); self.c=c
    def forward(self, x): return x[:, :, :-self.c].contiguous() if self.c>0 else x

class TemporalBlock(nn.Module):
    def __init__(self, n_in, n_out, k, dilation, padding, dropout=0.0):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv1d(n_in, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
            nn.Conv1d(n_out, n_out, k, padding=padding, dilation=dilation),
            nn.ReLU(), nn.Dropout(dropout), Chomp1d(padding),
        )
        self.down = nn.Conv1d(n_in, n_out, 1) if n_in!=n_out else nn.Identity()
    def forward(self, x):
        out = self.seq(x)
        return out + self.down(x)

class TCNClassifier(nn.Module):
    def __init__(self, feats=34, channels=(64,128,256), k=7, dropout=0.0, num_classes=10):
        super().__init__()
        layers=[]; n_in=feats
        for i, ch in enumerate(channels):
            dil = 2**i
            pad = (k-1)*dil
            layers.append(TemporalBlock(n_in, ch, k, dil, pad, dropout))
            n_in = ch
        self.tcn = nn.Sequential(*layers)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc   = nn.Linear(n_in, num_classes)
    def forward(self, x):                # x: (B,T,F)
        x = x.transpose(1,2)             # (B,F,T)
        y = self.tcn(x)                  # (B,C,T)
        y = self.pool(y).squeeze(-1)     # (B,C)
        return self.fc(y)

# ------------------- Seq utilities --------------------
def pad_or_center_trim(x: np.ndarray, T: int) -> np.ndarray:
    t = len(x)
    if t == T: return x
    if t > T:
        s = (t - T)//2
        return x[s:s+T]
    pad = np.zeros((T - t, x.shape[1]), dtype=x.dtype)
    return np.concatenate([x, pad], axis=0)

# ---------- Optional: video → YOLOv8 pose seq ----------
def extract_pose_sequence_yolo_batched(
    video_path: Path, every_n=3, batch_size=32, imgsz=448, conf=0.25, device="cpu"
):
    """
    Optional. Requires: pip install ultralytics opencv-python-headless
    Produces normalized 17x2 sequences → (T,34)
    """
    import cv2
    from ultralytics import YOLO

    LEFT_HIP, RIGHT_HIP = 11, 12
    LEFT_SHO, RIGHT_SHO = 5, 6

    def _pre(frame, target=448):
        h, w = frame.shape[:2]
        if max(h, w) > target:
            s = target / max(h, w)
            frame = cv2.resize(frame, (int(w*s), int(h*s)), interpolation=cv2.INTER_AREA)
        return frame

    pose_model = YOLO("yolov8n-pose.pt")
    if torch.cuda.is_available() and device != "cpu":
        pose_model.to(0)
        try: pose_model.model.half()
        except: pass

    cap = cv2.VideoCapture(str(video_path))
    frames=[]; fidx=0
    while cap.isOpened():
        ok, frame = cap.read()
        if not ok: break
        if fidx % every_n == 0:
            frames.append(_pre(frame, imgsz))
        fidx += 1
    cap.release()

    if not frames:
        return np.zeros((0,34), dtype=np.float32)

    seq=[]
    for i in range(0, len(frames), batch_size):
        batch = frames[i:i+batch_size]
        results = pose_model.predict(
            batch, imgsz=imgsz, device=0 if (torch.cuda.is_available() and device!="cpu") else device,
            conf=conf, max_det=1, verbose=False
        )
        for r in results:
            if not hasattr(r,"keypoints") or r.keypoints is None or len(r.keypoints)==0:
                continue
            xy = r.keypoints.xy[0].float().cpu().numpy()  # (17,2)
            origin = (xy[LEFT_HIP] + xy[RIGHT_HIP]) / 2.0
            shoulder_mid = (xy[LEFT_SHO] + xy[RIGHT_SHO]) / 2.0
            scale = np.linalg.norm(shoulder_mid - origin) or 1.0
            xy_norm = (xy - origin) / scale
            seq.append(xy_norm.reshape(-1))
    if not seq:
        return np.zeros((0,34), dtype=np.float32)
    return np.asarray(seq, dtype=np.float32)

# ----------------- Angles + feedback -------------------
LHIP,RHIP,LKNEE,RKNEE,LANK,RANK = 11,12,13,14,15,16
LSHO,RSHO,LELB,RELB,LWR,RWR     = 5,6,7,8,9,10

def angle_deg(a, b, c):
    ba = a - b; bc = c - b
    den = (np.linalg.norm(ba)*np.linalg.norm(bc))
    if den == 0: return 180.0
    cosv = np.clip(np.dot(ba,bc)/den, -1.0, 1.0)
    return float(np.degrees(np.arccos(cosv)))

def angles_from_seq(seq_xy: np.ndarray):
    stats = {"knee_r":[], "knee_l":[], "elbow_r":[], "elbow_l":[]}
    for t in range(len(seq_xy)):
        xy = seq_xy[t].reshape(17,2)
        stats["knee_r"].append(angle_deg(xy[RHIP], xy[RKNEE], xy[RANK]))
        stats["knee_l"].append(angle_deg(xy[LHIP], xy[LKNEE], xy[LANK]))
        stats["elbow_r"].append(angle_deg(xy[RSHO], xy[RELB], xy[RWR]))
        stats["elbow_l"].append(angle_deg(xy[LSHO], xy[LELB], xy[LWR]))
    return {k: {
        "min": float(np.min(v)), "max": float(np.max(v)),
        "p10": float(np.percentile(v,10)), "p90": float(np.percentile(v,90))
    } for k,v in stats.items()}

def feedback_from_stats(label: str, stats: dict):
    L = (label or "").lower()
    fb=[]
    # Squat / hinge-ish cues
    if any(s in L for s in ["squat","leg extension","hip thrust"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min > 100: fb.append("Depth shallow — aim near ~90° knee flexion.")
        elif knee_min < 60: fb.append("Very deep — ensure control and comfort.")
        else: fb.append("Good depth.")
    if any(s in L for s in ["deadlift","romanian"]):
        knee_min = min(stats["knee_l"]["min"], stats["knee_r"]["min"])
        if knee_min < 70: fb.append("Knees bending a lot — hinge more from hips.")
    # Push/press elbow flare
    if any(s in L for s in ["push","press","bench"]):
        emax = max(stats["elbow_l"]["p90"], stats["elbow_r"]["p90"])
        if emax > 130: fb.append("Elbows flaring — target ~45° to protect shoulders.")
        else: fb.append("Elbow tracking looks reasonable.")
    # Curl ROM
    if "curl" in L:
        emin = min(stats["elbow_l"]["min"], stats["elbow_r"]["min"])
        if emin > 80: fb.append("Limited elbow flexion — curl through fuller ROM.")
    if "plank" in L:
        fb.append("Keep straight line head→heels; avoid hip sag.")
    if not fb:
        fb.append("Form looks okay; review depth and control.")
    return fb

# -------------- Sliding-window (no TTA) ----------------
@torch.no_grad()
def logits_on_seq(model, seq, device):
    x = torch.tensor(seq, dtype=torch.float32, device=device).unsqueeze(0)
    return model(x).squeeze(0).cpu().numpy()

def vote_over_windows(model, seq_full, device, seq_len=90, hop=30):
    T = len(seq_full)
    if T <= seq_len:
        return logits_on_seq(model, pad_or_center_trim(seq_full, seq_len), device)
    logits_sum = None; count=0
    for start in range(0, max(1, T - seq_len + 1), hop):
        chunk = seq_full[start:start+seq_len]
        if len(chunk) < seq_len:
            chunk = pad_or_center_trim(chunk, seq_len)
        L = logits_on_seq(model, chunk, device)
        logits_sum = L if logits_sum is None else (logits_sum + L)
        count += 1
    return logits_sum / max(count,1)

# ---------------------- Labels helper -------------------
def load_labels(cache_dir: Path, fallback_n=None):
    labels_json = Path(cache_dir)/"labels.json"
    if labels_json.exists():
        meta = json.loads(Path(labels_json).read_text())
        labs = meta.get("labels", [])
        if labs: return labs
    # fallback generate
    if fallback_n is not None:
        return [f"class_{i:02d}" for i in range(fallback_n)]
    # infer from file prefixes if present
    ids=[]
    for f in Path(cache_dir).glob("*.npy"):
        m = re.match(r"^(\d+?)__.+?\.npy$", f.name)
        if m: ids.append(int(m.group(1)))
    return [f"class_{i:02d}" for i in range(max(ids)+1)] if ids else []

# --------------------------- Main -----------------------
def main():
    ap = argparse.ArgumentParser("Baseline TCN inference + feedback (no TTA, no focal)")
    ap.add_argument("--checkpoint", type=Path, required=True, help="baseline TCN .pt or raw state_dict")
    ap.add_argument("--cache_dir", type=Path, required=True, help="dir with labels.json from training")
    ap.add_argument("--npy", type=Path, default=None, help="pre-extracted pose sequence (.npy)")
    ap.add_argument("--video", type=Path, default=None, help="optional video path (requires ultralytics+opencv)")
    ap.add_argument("--seq_len", type=int, default=90)
    ap.add_argument("--hop", type=int, default=30)
    ap.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
    # YOLO flags (only if using --video)
    ap.add_argument("--every_n", type=int, default=3)
    ap.add_argument("--imgsz", type=int, default=448)
    ap.add_argument("--conf", type=float, default=0.25)
    ap.add_argument("--batch_size", type=int, default=32)
    args = ap.parse_args()

    print("[infer] args:", vars(args))  # sanity log

    # Load model
    obj = torch.load(args.checkpoint, map_location=args.device)
    if isinstance(obj, dict) and "state_dict" in obj and "hparams" in obj:
        hp = obj["hparams"]
        channels = tuple(hp.get("channels", (64,128,256)))
        kernel   = int(hp.get("kernel", 7))
        num_classes = int(hp.get("num_classes", 22))
        labels = obj.get("labels") or load_labels(args.cache_dir, fallback_n=num_classes)
        model = TCNClassifier(feats=34, channels=channels, k=kernel, dropout=0.0, num_classes=len(labels)).to(args.device)
        model.load_state_dict(obj["state_dict"], strict=True)
    else:
        # raw state_dict: use baseline defaults (adjust if your training used different ones)
        labels = load_labels(args.cache_dir)
        model = TCNClassifier(feats=34, channels=(64,128,256), k=7, dropout=0.0, num_classes=len(labels)).to(args.device)
        model.load_state_dict(obj, strict=False)
        print("[warn] Loaded raw state_dict with default channels=(64,128,256), kernel=7; ensure these match training.")
    model.eval()

    # Get sequence
    if args.npy:
        if not args.npy.exists():
            raise SystemExit(f"npy not found: {args.npy}")
        seq_full = np.load(args.npy)
    elif args.video:
        if not args.video.exists():
            raise SystemExit(f"video not found: {args.video}")
        print("[info] extracting pose from video (YOLOv8n-pose)...")
        seq_full = extract_pose_sequence_yolo_batched(
            args.video, every_n=args.every_n, batch_size=args.batch_size,
            imgsz=args.imgsz, conf=args.conf, device=args.device
        )
    else:
        raise SystemExit("Provide either --npy (recommended) or --video.")

    if len(seq_full)==0:
        raise SystemExit("Empty sequence. Check video visibility/pose extraction or npy path.")

    # Predict (sliding-window vote; no TTA)
    logits = vote_over_windows(model, seq_full, args.device, seq_len=args.seq_len, hop=args.hop)
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    pred_id = int(np.argmax(probs))
    pred_label = labels[pred_id] if 0 <= pred_id < len(labels) else str(pred_id)

    print("\n=== Inference (baseline TCN) ===")
    print(f"Predicted class: {pred_label} (id={pred_id})")
    topk = min(5, len(labels))
    order = np.argsort(-probs)[:topk]
    for i in order:
        print(f"  {labels[i]:22s} p={probs[i]:.3f}")

    # Feedback (angles on center-trimmed window)
    seq = pad_or_center_trim(seq_full, args.seq_len)
    stats = angles_from_seq(seq)
    cues = feedback_from_stats(pred_label, stats)
    print("\n=== Angle stats ===")
    print(json.dumps(stats, indent=2))
    print("\n=== Feedback ===")
    for c in cues: print(" -", c)

if __name__ == "__main__":
    main()

Overwriting infer_baseline.py


In [ ]:
!python -u infer_baseline.py \
  --checkpoint /content/best_tcn.pt \
  --cache_dir /content/pose_seq_yolo \
  --npy "/content/pose_seq_yolo/00__barbell biceps curl_11.npy" \
  --seq_len 90 --hop 30

[infer] args: {'checkpoint': PosixPath('/content/best_tcn.pt'), 'cache_dir': PosixPath('/content/pose_seq_yolo'), 'npy': PosixPath('/content/pose_seq_yolo/00__barbell biceps curl_11.npy'), 'video': None, 'seq_len': 90, 'hop': 30, 'device': 'cpu', 'every_n': 3, 'imgsz': 448, 'conf': 0.25, 'batch_size': 32}

=== Inference (baseline TCN) ===
Predicted class: squat (id=18)
  squat                  p=0.174
  hammer curl            p=0.136
  barbell biceps curl    p=0.102
  incline bench press    p=0.082
  tricep Pushdown        p=0.072

=== Angle stats ===
{
  "knee_r": {
    "min": 151.62733459472656,
    "max": 180.0,
    "p10": 156.50329132080077,
    "p90": 180.0
  },
  "knee_l": {
    "min": 149.34751892089844,
    "max": 180.0,
    "p10": 152.1273162841797,
    "p90": 180.0
  },
  "elbow_r": {
    "min": 1.5356959104537964,
    "max": 180.0,
    "p10": 47.84908218383789,
    "p90": 180.0
  },
  "elbow_l": {
    "min": 45.768863677978516,
    "max": 180.0,
    "p10": 76.17696914672852,

In [ ]:
!python -u infer_baseline.py \
  --checkpoint /content/best_tcn.pt \
  --cache_dir /content/pose_seq_yolo \
  --video /content/m2-res_640p.mp4 \
  --seq_len 90 --hop 30 \
  --every_n 3 --imgsz 448 --conf 0.25

[infer] args: {'checkpoint': PosixPath('/content/best_tcn.pt'), 'cache_dir': PosixPath('/content/pose_seq_yolo'), 'npy': None, 'video': PosixPath('/content/m2-res_640p.mp4'), 'seq_len': 90, 'hop': 30, 'device': 'cpu', 'every_n': 3, 'imgsz': 448, 'conf': 0.25, 'batch_size': 32}
[info] extracting pose from video (YOLOv8n-pose)...

=== Inference (baseline TCN) ===
Predicted class: push-up (id=14)
  push-up                p=0.383
  plank                  p=0.325
  decline bench press    p=0.036
  romanian deadlift      p=0.028
  shoulder press         p=0.022

=== Angle stats ===
{
  "knee_r": {
    "min": 53.70746994018555,
    "max": 168.45713806152344,
    "p10": 74.53679809570312,
    "p90": 139.53519134521486
  },
  "knee_l": {
    "min": 12.006741523742676,
    "max": 154.3284912109375,
    "p10": 49.254337310791016,
    "p90": 118.75203781127931
  },
  "elbow_r": {
    "min": 68.95397186279297,
    "max": 159.97483825683594,
    "p10": 81.73776321411133,
    "p90": 154.5933883666992

In [ ]:
!python -u infer_baseline.py \
  --checkpoint /content/best_tcn.pt \
  --cache_dir /content/pose_seq_yolo \
  --video /content/m2-res_640p.mp4 \
  --seq_len 90 --hop 30 \
  --every_n 3 --imgsz 448 --conf 0.25

[infer] args: {'checkpoint': PosixPath('/content/best_tcn.pt'), 'cache_dir': PosixPath('/content/pose_seq_yolo'), 'npy': None, 'video': PosixPath('/content/m2-res_640p.mp4'), 'seq_len': 90, 'hop': 30, 'device': 'cpu', 'every_n': 3, 'imgsz': 448, 'conf': 0.25, 'batch_size': 32}
[info] extracting pose from video (YOLOv8n-pose)...

=== Inference (baseline TCN) ===
Predicted class: push-up (id=14)
  push-up                p=0.383
  plank                  p=0.325
  decline bench press    p=0.036
  romanian deadlift      p=0.028
  shoulder press         p=0.022

=== Angle stats ===
{
  "knee_r": {
    "min": 53.70746994018555,
    "max": 168.45713806152344,
    "p10": 74.53679809570312,
    "p90": 139.53519134521486
  },
  "knee_l": {
    "min": 12.006741523742676,
    "max": 154.3284912109375,
    "p10": 49.254337310791016,
    "p90": 118.75203781127931
  },
  "elbow_r": {
    "min": 68.95397186279297,
    "max": 159.97483825683594,
    "p10": 81.73776321411133,
    "p90": 154.5933883666992

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ==== CONFIG ====
CKPT_PATH   = "/content/best_tcn.pt"        # your trained baseline TCN
CACHE_DIR   = "/content/pose_seq_yolo"      # where the .npy sequences live
SAVE_ROOT   = "/content/drive/MyDrive/fitness_form_ai"  # change if you like

# Subfolders in Drive
SAVE_MODELS = f"{SAVE_ROOT}/models"
SAVE_CACHE  = f"{SAVE_ROOT}/pose_cache"
SAVE_META   = f"{SAVE_ROOT}/meta"

import os
for p in [SAVE_ROOT, SAVE_MODELS, SAVE_CACHE, SAVE_META]:
    os.makedirs(p, exist_ok=True)
print("Drive folders ready.")

Mounted at /content/drive
Drive folders ready.


In [ ]:
import os, json, shutil, hashlib
from pathlib import Path
from datetime import datetime

ckpt_src  = Path(CKPT_PATH)
cache_src = Path(CACHE_DIR)
labels_src = cache_src / "labels.json"

assert ckpt_src.exists(), f"Missing checkpoint: {ckpt_src}"
assert cache_src.exists(), f"Missing cache dir: {cache_src}"

# Copy checkpoint with timestamped name (and also keep a stable name)
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
ckpt_dst1 = Path(SAVE_MODELS) / f"best_tcn_{stamp}.pt"
ckpt_dst2 = Path(SAVE_MODELS) / "best_tcn.pt"   # stable pointer

shutil.copy2(ckpt_src, ckpt_dst1)
shutil.copy2(ckpt_src, ckpt_dst2)

# Copy labels.json (if present)
if labels_src.exists():
    shutil.copy2(labels_src, Path(SAVE_CACHE) / "labels.json")

# Copy all .npy pose sequences (skip if already identical)
def sha256sum(p):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for chunk in iter(lambda: f.read(1024*1024), b""):
            h.update(chunk)
    return h.hexdigest()

copied = 0
skipped = 0
for npy in cache_src.glob("*.npy"):
    dst = Path(SAVE_CACHE) / npy.name
    if dst.exists():
        # skip if same content
        if sha256sum(npy) == sha256sum(dst):
            skipped += 1
            continue
    shutil.copy2(npy, dst)
    copied += 1

# Write a manifest with basic info
manifest = {
    "saved_at": stamp,
    "ckpt_dst": str(ckpt_dst1),
    "ckpt_alias": str(ckpt_dst2),
    "cache_dir": str(SAVE_CACHE),
    "counts": {
        "npy_on_colab": len(list(cache_src.glob("*.npy"))),
        "npy_on_drive": len(list(Path(SAVE_CACHE).glob("*.npy")))
    }
}
with open(Path(SAVE_META)/f"manifest_{stamp}.json", "w") as f:
    json.dump(manifest, f, indent=2)

print(f"Copied: {copied} .npy | Skipped (already identical): {skipped}")
print(f"Model saved to: {ckpt_dst1} and {ckpt_dst2}")
print(f"Manifest: {(Path(SAVE_META)/f'manifest_{stamp}.json')}")

✅ Copied: 630 .npy | Skipped (already identical): 0
✅ Model saved to: /content/drive/MyDrive/fitness_form_ai/models/best_tcn_20250826-070817.pt and /content/drive/MyDrive/fitness_form_ai/models/best_tcn.pt
📄 Manifest: /content/drive/MyDrive/fitness_form_ai/meta/manifest_20250826-070817.json


In [ ]:
import shutil, os
zip_base = f"{SAVE_ROOT}/pose_cache_snapshot"
zip_path = shutil.make_archive(zip_base, "zip", SAVE_CACHE)
print("ZIP written:", zip_path)

ZIP written: /content/drive/MyDrive/fitness_form_ai/pose_cache_snapshot.zip
